# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

## A prototype for a chatbot to answer questions from students about a class.

In [ ]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
# Initialization

load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

In [ ]:
system_message = "You are a helpful assistant for a mathematics class at Valparaiso University. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

In [ ]:
exam_dates = {"midterm": "February 27, 2025", "final": "may 6, 2025"}

def get_exam_date(exam_type):
    print(f"Tool get_exam_date called for {exam_type}")
    exam = exam_type.lower()
    return exam_dates.get(exam_type, "Unknown")

In [ ]:
# There's a particular dictionary structure that's required to describe our function:

exam_function = {
    "name": "get_exam_date",
    "description": "Get the exam date for the class. Call this whenever you need to know the date of an exam, for example when a customer asks 'When is the exam'",
    "parameters": {
        "type": "object",
        "properties": {
            "exam_type": {
                "type": "string",
                "description": "The type of exam the student wants to date of",
            },
        },
        "required": ["exam_type"],
        "additionalProperties": False
    }
}

In [ ]:
tools = [{"type": "function", "function": exam_function}]

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, my_type = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [ ]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    my_type = arguments.get('exam_type')
    date = get_exam_date(my_type)
    response = {
        "role": "tool",
        "content": json.dumps({"exam_type": my_type,"date": date}),
        "tool_call_id": tool_call.id
    }
    return response, my_type

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch(share=True)